In [1]:
#--Dependencies--#

#-- Data Cleaning Libraries:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

#-- Data Visualization Libraries:
from matplotlib import pyplot as plt
import seaborn as sns #--just in case we need it, probably won't


#-- Web Scraping Libraries:
import os
import time
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser

#The best progress bar
from tqdm import tqdm_notebook


In [2]:
raw_df = pd.read_csv("Players_birth_raw.csv")


In [3]:
#Create function to take out year

def year_func(row):
    year = row[-5: -1]
    
    return year

In [4]:
raw_df["Year"]= raw_df["1-Jan"].apply(lambda x : year_func(x))

In [5]:
#normalize year

def name_func(row):
    name = row[0:-6]
    
    return name

In [6]:
raw_df["1-Jan"] = raw_df["1-Jan"].apply(lambda row: name_func(row))


In [7]:
raw_df = raw_df.rename(columns={"1-Jan": "Player"})


In [8]:
raw_df = raw_df[raw_df.Player != '']

In [9]:
#Player index function

def play_index(row):
    player = row.Player
    year = row.year
    
    index = player + "_" + year
    
    return index

In [10]:
raw_df["Player Index"] = raw_df.Player + "_" + raw_df.Year

In [11]:
raw_df["Year"] = raw_df["Year"].apply(lambda x: int(x))

In [12]:
raw_df = raw_df[raw_df.Year >= 1954]

In [13]:
raw_df.Player = raw_df.Player.apply(lambda x: x.strip())

In [14]:
pindex_df = raw_df[["Player", "Player Index"]]

In [15]:
nba_df[nba_df["Player"] == "Michael Jordan*"]

NameError: name 'nba_df' is not defined

In [16]:
nba_df = pd.read_csv("NBA_agg.csv", header=0)

In [17]:
nba_df.head()

,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
0,Michael Jordan*,37.970000,10.960000,22.810000,0.479700,0.730000,2.010000,0.332900,10.240000,20.800000,...,1.390000,4.890000,6.300,4.9800,2.0900,0.660000,2.430000,2.360000,28.630000,39.764000
1,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,15.368750,...,1.206250,6.243750,7.425,7.2375,1.6125,0.756250,3.481250,1.837500,27.156250,38.455000
2,Kevin Durant,36.616667,9.175000,18.566667,0.495167,1.900000,4.950000,0.379667,7.283333,13.633333,...,0.708333,6.358333,7.075,4.1250,1.1000,1.116667,3.091667,1.883333,26.883333,37.046667
3,Karl Malone*,37.366667,9.033333,17.546667,0.513200,0.046667,0.233333,0.233933,8.973333,17.306667,...,2.220000,7.773333,9.980,3.8800,1.4000,0.786667,2.946667,2.933333,25.046667,35.304000
4,James Harden,34.080000,7.150000,16.090000,0.443400,2.630000,7.220000,0.366400,4.520000,8.890000,...,0.750000,4.440000,5.190,6.1500,1.5500,0.490000,3.600000,2.570000,24.220000,35.806000


In [18]:
#clean nba_df

def astriks (row):
    name = row
    
    if name[-1] == "*":
        name = name[0:-1]
        return name
    else:
        return name
    

In [19]:
nba_df["Player"] = nba_df["Player"].apply(lambda x: astriks(x))

In [20]:
nba_df.head()

,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
0,Michael Jordan,37.970000,10.960000,22.810000,0.479700,0.730000,2.010000,0.332900,10.240000,20.800000,...,1.390000,4.890000,6.300,4.9800,2.0900,0.660000,2.430000,2.360000,28.630000,39.764000
1,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,15.368750,...,1.206250,6.243750,7.425,7.2375,1.6125,0.756250,3.481250,1.837500,27.156250,38.455000
2,Kevin Durant,36.616667,9.175000,18.566667,0.495167,1.900000,4.950000,0.379667,7.283333,13.633333,...,0.708333,6.358333,7.075,4.1250,1.1000,1.116667,3.091667,1.883333,26.883333,37.046667
3,Karl Malone,37.366667,9.033333,17.546667,0.513200,0.046667,0.233333,0.233933,8.973333,17.306667,...,2.220000,7.773333,9.980,3.8800,1.4000,0.786667,2.946667,2.933333,25.046667,35.304000
4,James Harden,34.080000,7.150000,16.090000,0.443400,2.630000,7.220000,0.366400,4.520000,8.890000,...,0.750000,4.440000,5.190,6.1500,1.5500,0.490000,3.600000,2.570000,24.220000,35.806000


In [21]:

agg_df = nba_df.merge(pindex_df, on="Player")

In [22]:
agg_df

,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,Player Index
0,Michael Jordan,37.970000,10.960000,22.810000,0.479700,0.730000,2.010000,0.332900,10.240000,20.800000,...,4.890000,6.300000,4.980000,2.090000,0.660000,2.430000,2.360000,28.630000,39.764000,Michael Jordan _1963
1,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,15.368750,...,6.243750,7.425000,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,LeBron James _1984
2,Kevin Durant,36.616667,9.175000,18.566667,0.495167,1.900000,4.950000,0.379667,7.283333,13.633333,...,6.358333,7.075000,4.125000,1.100000,1.116667,3.091667,1.883333,26.883333,37.046667,Kevin Durant _1988
3,Karl Malone,37.366667,9.033333,17.546667,0.513200,0.046667,0.233333,0.233933,8.973333,17.306667,...,7.773333,9.980000,3.880000,1.400000,0.786667,2.946667,2.933333,25.046667,35.304000,Karl Malone _1963
4,James Harden,34.080000,7.150000,16.090000,0.443400,2.630000,7.220000,0.366400,4.520000,8.890000,...,4.440000,5.190000,6.150000,1.550000,0.490000,3.600000,2.570000,24.220000,35.806000,James Harden _1989
5,Kobe Bryant,35.770000,8.450000,19.040000,0.441400,1.300000,4.060000,0.317700,7.150000,14.975000,...,4.120000,5.200000,4.760000,1.410000,0.460000,3.130000,2.440000,24.200000,33.306000,Kobe Bryant _1978
6,Allen Iverson,39.330000,8.365000,19.540000,0.433700,0.965000,3.100000,0.341050,7.405000,16.445000,...,2.660000,3.375000,5.895000,1.855000,0.155000,3.415000,1.785000,24.160000,30.572500,Allen Iverson _1975
7,Carmelo Anthony,35.511111,8.483333,18.933333,0.445889,1.344444,3.833333,0.339722,7.138889,15.094444,...,4.894444,6.544444,2.883333,0.977778,0.511111,2.561111,2.900000,23.761111,32.598333,Carmelo Anthony _1984
8,Damian Lillard,36.257143,7.800000,18.042857,0.433286,2.785714,7.471429,0.368571,5.085714,10.542857,...,3.542857,4.171429,6.357143,0.985714,0.328571,2.771429,2.028571,23.642857,33.954286,Damian Lillard _1990
9,Anthony Davis,34.442857,8.871429,17.128571,0.517000,0.385714,1.257143,0.228429,8.471429,15.914286,...,7.914286,10.514286,2.142857,1.385714,2.385714,1.857143,2.385714,23.571429,38.901429,Anthony Davis _1993


In [23]:
cols = list(agg_df.columns)

cols.insert(0, cols.pop(cols.index('Player Index')))

In [24]:
agg_df = agg_df.reindex(columns= cols)

In [25]:
agg_df.head()

,Player Index,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
0,Michael Jordan _1963,Michael Jordan,37.970000,10.960000,22.810000,0.479700,0.730000,2.010000,0.332900,10.240000,...,1.390000,4.890000,6.300,4.9800,2.0900,0.660000,2.430000,2.360000,28.630000,39.764000
1,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,1.206250,6.243750,7.425,7.2375,1.6125,0.756250,3.481250,1.837500,27.156250,38.455000
2,Kevin Durant _1988,Kevin Durant,36.616667,9.175000,18.566667,0.495167,1.900000,4.950000,0.379667,7.283333,...,0.708333,6.358333,7.075,4.1250,1.1000,1.116667,3.091667,1.883333,26.883333,37.046667
3,Karl Malone _1963,Karl Malone,37.366667,9.033333,17.546667,0.513200,0.046667,0.233333,0.233933,8.973333,...,2.220000,7.773333,9.980,3.8800,1.4000,0.786667,2.946667,2.933333,25.046667,35.304000
4,James Harden _1989,James Harden,34.080000,7.150000,16.090000,0.443400,2.630000,7.220000,0.366400,4.520000,...,0.750000,4.440000,5.190,6.1500,1.5500,0.490000,3.600000,2.570000,24.220000,35.806000


In [32]:
#import NBA_clean

NBA_df = pd.read_csv("NBA_clean.csv")


NBA_df.columns

Index(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%',
       'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', 'PER'],
      dtype='object')

In [33]:
#get teams for each player

player_team = NBA_df[["Player", "Tm"]]

player_team.head()

,Player,Tm
0,Mark Acres,ORL
1,Michael Adams,DEN
2,Mark Aguirre,DET
3,Danny Ainge,SAC
4,Mark Alarie,WSB


In [44]:
#merge on agg_df

agg_df.merge(player_team, on="Player", how= "left")

,Player Index,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,Tm
0,Michael Jordan _1963,Michael Jordan,37.970000,10.960000,22.810000,0.479700,0.730000,2.010000,0.332900,10.240000,...,4.890000,6.300,4.980000,2.090000,0.660000,2.430000,2.360000,28.630000,39.764000,NaN
1,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,CLE
2,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,CLE
3,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,CLE
4,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,CLE
5,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,CLE
6,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,CLE
7,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,CLE
8,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,MIA
9,LeBron James _1984,LeBron James,38.493750,9.881250,19.587500,0.506375,1.450000,4.206250,0.343500,8.437500,...,6.243750,7.425,7.237500,1.612500,0.756250,3.481250,1.837500,27.156250,38.455000,MIA
